In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from datetime import datetime

# ==================== BANCO DE DADOS ====================

def criar_banco():
    """Cria o banco de dados e a tabela de estoque se não existir"""
    conn = sqlite3.connect("estoque.db")
    cursor = conn.cursor()
    
    # Verifica se a tabela existe e tem a estrutura correta
    try:
        cursor.execute("PRAGMA table_info(estoque)")
        colunas = cursor.fetchall()
        
        # Se a tabela existe mas não tem 6 colunas, recria
        if colunas and len(colunas) != 6:
            cursor.execute("DROP TABLE IF EXISTS estoque")
    except:
        pass
    
    # Cria a tabela com a estrutura correta
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS estoque (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT NOT NULL,
            tipo TEXT NOT NULL,
            quantidade INTEGER NOT NULL,
            data_adicao TEXT NOT NULL,
            ultima_atualizacao TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()

def adicionar_item_bd(nome, tipo, quantidade):
    """Adiciona ou atualiza um item no banco de dados"""
    conn = sqlite3.connect("estoque.db")
    cursor = conn.cursor()
    
    # Verifica se o item já existe
    cursor.execute("SELECT id, quantidade FROM estoque WHERE nome = ?", (nome,))
    resultado = cursor.fetchone()
    
    data_hora_atual = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    
    if resultado:
        # Atualiza a quantidade
        nova_quantidade = resultado[1] + quantidade
        cursor.execute("""
            UPDATE estoque 
            SET quantidade = ?, ultima_atualizacao = ?, tipo = ?
            WHERE id = ?
        """, (nova_quantidade, data_hora_atual, tipo, resultado[0]))
    else:
        # Insere novo item
        cursor.execute("""
            INSERT INTO estoque (nome, tipo, quantidade, data_adicao, ultima_atualizacao)
            VALUES (?, ?, ?, ?, ?)
        """, (nome, tipo, quantidade, data_hora_atual, data_hora_atual))
    
    conn.commit()
    conn.close()

def remover_item_bd(nome, quantidade):
    """Remove quantidade de um item do banco de dados"""
    conn = sqlite3.connect("estoque.db")
    cursor = conn.cursor()
    
    cursor.execute("SELECT id, quantidade FROM estoque WHERE nome = ?", (nome,))
    resultado = cursor.fetchone()
    
    if not resultado:
        conn.close()
        return False, "Item não encontrado no estoque!"
    
    if resultado[1] < quantidade:
        conn.close()
        return False, "Quantidade para remover é maior que o disponível!"
    
    nova_quantidade = resultado[1] - quantidade
    data_hora_atual = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    
    if nova_quantidade == 0:
        cursor.execute("DELETE FROM estoque WHERE id = ?", (resultado[0],))
    else:
        cursor.execute("""
            UPDATE estoque 
            SET quantidade = ?, ultima_atualizacao = ?
            WHERE id = ?
        """, (nova_quantidade, data_hora_atual, resultado[0]))
    
    conn.commit()
    conn.close()
    return True, "Item removido com sucesso!"

def obter_estoque(termo_busca="", campo_busca="nome"):
    """Retorna itens do estoque, opcionalmente filtrados por termo e campo"""
    conn = sqlite3.connect("estoque.db")
    cursor = conn.cursor()
    
    sql = "SELECT * FROM estoque"
    params = []
    
    if termo_busca:
        # Lógica de busca por nome, tipo ou data_adicao
        if campo_busca == "nome":
            sql += " WHERE nome LIKE ?"
            params.append(f"%{termo_busca}%")
        elif campo_busca == "tipo":
            sql += " WHERE tipo LIKE ?"
            params.append(f"%{termo_busca}%")
        elif campo_busca == "data_adicao":
            # Busca por data de adição (DD/MM/AAAA)
            sql += " WHERE data_adicao LIKE ?"
            params.append(f"%{termo_busca}%")
            
    sql += " ORDER BY nome"
    
    cursor.execute(sql, tuple(params))
    itens = cursor.fetchall()
    conn.close()
    return itens

def obter_estatisticas():
    """Retorna estatísticas do estoque"""
    conn = sqlite3.connect("estoque.db")
    cursor = conn.cursor()
    
    cursor.execute("SELECT COUNT(*), SUM(quantidade) FROM estoque")
    total_tipos, total_itens = cursor.fetchone()
    
    cursor.execute("SELECT tipo, COUNT(*) FROM estoque GROUP BY tipo")
    por_tipo = cursor.fetchall()
    
    conn.close()
    return {
        'total_tipos': total_tipos or 0,
        'total_itens': total_itens or 0,
        'por_tipo': por_tipo or []
    }

# ==================== INTERFACE GRÁFICA ====================

class GerenciadorEstoque:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistema de Gerenciamento de Estoque")
        self.root.geometry("800x600")
        self.root.configure(bg="#F4F4F4")
        
        # Criar banco de dados
        criar_banco()
        
        # Criar notebook (abas)
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Criar as abas
        self.criar_aba_inicio()
        self.criar_aba_gerenciamento()
        self.criar_aba_relatorios()
        
        # Atualizar dados iniciais
        self.atualizar_estatisticas()
    
    # ==================== ABA INÍCIO ====================
    
    def criar_aba_inicio(self):
        """Cria a aba de início com resumo do estoque"""
        aba_inicio = ttk.Frame(self.notebook)
        self.notebook.add(aba_inicio, text="  🏠 Início  ")
        
        # Frame principal
        frame_principal = tk.Frame(aba_inicio, bg="#ffffff")
        frame_principal.pack(fill='both', expand=True, padx=20, pady=20)
        
        # Título
        titulo = tk.Label(
            frame_principal,
            text="📦 Sistema de Gerenciamento de Estoque",
            font=("Arial", 24, "bold"),
            bg="#ffffff",
            fg="#333333"
        )
        titulo.pack(pady=20)
        
        # Subtítulo
        subtitulo = tk.Label(
            frame_principal,
            text="Bem-vindo ao sistema de controle de estoque",
            font=("Arial", 12),
            bg="#ffffff",
            fg="#7f8c8d"
        )
        subtitulo.pack(pady=5)
        
        # Frame de estatísticas
        frame_stats = tk.Frame(frame_principal, bg="#ffffff")
        frame_stats.pack(pady=30)
        
        # Card 1 - Total de Tipos
        self.card_tipos = self.criar_card_estatistica(
            frame_stats, "Total de Produtos", "0", "#3498db", 0
        )
        
        # Card 2 - Total de Itens
        self.card_itens = self.criar_card_estatistica(
            frame_stats, "Total de Unidades", "0", "#2ecc71", 1
        )
        
        # Informações adicionais
        frame_info = tk.Frame(frame_principal, bg="#EAEAEA", relief=tk.RIDGE, bd=2)
        frame_info.pack(pady=20, padx=40, fill='x')
        
        info_text = """
        📌 Funcionalidades do Sistema:
        
        • Adicionar novos produtos ao estoque
        • Remover produtos do estoque
        • Visualizar relatórios detalhados
        • Acompanhar data e hora de movimentações
        • Categorizar produtos por tipo
        """
        
        label_info = tk.Label(
            frame_info,
            text=info_text,
            font=("Arial", 10),
            bg="#EAEAEA",
            fg="#333333",
            justify=tk.LEFT
        )
        label_info.pack(pady=15, padx=15)
        
        # Rodapé
        rodape = tk.Label(
            frame_principal,
            text="Desenvolvido por Kauã ",
            font=("Arial", 9),
            bg="#ffffff",
            fg="#95a5a6"
        )
        rodape.pack(side=tk.BOTTOM, pady=10)
    
    def criar_card_estatistica(self, parent, titulo, valor, cor, coluna):
        """Cria um card de estatística"""
        frame = tk.Frame(parent, bg=cor, relief=tk.RAISED, bd=3)
        frame.grid(row=0, column=coluna, padx=20, pady=10, ipadx=30, ipady=20)
        
        label_titulo = tk.Label(
            frame,
            text=titulo,
            font=("Arial", 12),
            bg=cor,
            fg="white"
        )
        label_titulo.pack(pady=5)
        
        label_valor = tk.Label(
            frame,
            text=valor,
            font=("Arial", 28, "bold"),
            bg=cor,
            fg="white"
        )
        label_valor.pack(pady=5)
        
        return label_valor
    
    # ==================== ABA GERENCIAMENTO ====================
    
    def criar_aba_gerenciamento(self):
        """Cria a aba de gerenciamento de estoque"""
        aba_gerenciamento = ttk.Frame(self.notebook)
        self.notebook.add(aba_gerenciamento, text="  📝 Gerenciamento  ")
        
        # Frame principal
        frame_principal = tk.Frame(aba_gerenciamento, bg="#ffffff")
        frame_principal.pack(fill='both', expand=True, padx=20, pady=20)
        
        # Título
        titulo = tk.Label(
            frame_principal,
            text="Gerenciamento de Estoque",
            font=("Arial", 18, "bold"),
            bg="#ffffff",
            fg="#333333"
        )
        titulo.pack(pady=15)
        
        # Frame de entrada de dados
        frame_entrada = tk.LabelFrame(
            frame_principal,
            text="  Informações do Produto  ",
            font=("Arial", 11, "bold"),
            bg="#EAEAEA",
            fg="#333333",
            padx=20,
            pady=15
        )
        frame_entrada.pack(pady=10, padx=20, fill='x')
        
        # Nome do produto
        tk.Label(
            frame_entrada,
            text="Nome do Produto:",
            font=("Arial", 10),
            bg="#ffffff"
        ).grid(row=0, column=0, sticky='w', pady=8)
        
        self.entry_nome = tk.Entry(frame_entrada, width=35, font=("Arial", 10))
        self.entry_nome.grid(row=0, column=1, pady=8, padx=10)
        
        # Tipo do produto
        tk.Label(
            frame_entrada,
            text="Tipo do Produto:",
            font=("Arial", 10),
            bg="#ffffff"
        ).grid(row=1, column=0, sticky='w', pady=8)
        
        self.entry_tipo = tk.Entry(frame_entrada, width=35, font=("Arial", 10))
        self.entry_tipo.grid(row=1, column=1, pady=8, padx=10)
        
        # Quantidade
        tk.Label(
            frame_entrada,
            text="Quantidade:",
            font=("Arial", 10),
            bg="#ffffff"
        ).grid(row=2, column=0, sticky='w', pady=8)
        
        self.entry_quantidade = tk.Entry(frame_entrada, width=35, font=("Arial", 10))
        self.entry_quantidade.grid(row=2, column=1, pady=8, padx=10)
        
        # Frame de botões
        frame_botoes = tk.Frame(frame_principal, bg="#ffffff")
        frame_botoes.pack(pady=15)
        
        btn_adicionar = tk.Button(
            frame_botoes,
            text="➕ Adicionar ao Estoque",
            font=("Arial", 11, "bold"),
            bg="#4CAF50",
            fg="white",
            width=20,
            height=2,
            cursor="hand2",
            command=self.adicionar_item
        )
        btn_adicionar.grid(row=0, column=0, padx=10)
        
        btn_remover = tk.Button(
            frame_botoes,
            text="➖ Remover do Estoque",
            font=("Arial", 11, "bold"),
            bg="#E53935",
            fg="white",
            width=20,
            height=2,
            cursor="hand2",
            command=self.remover_item
        )
        btn_remover.grid(row=0, column=1, padx=10)
        
        btn_limpar = tk.Button(
            frame_botoes,
            text="🔄 Limpar Campos",
            font=("Arial", 11, "bold"),
            bg="#9E9E9E",
            fg="white",
            width=20,
            height=2,
            cursor="hand2",
            command=self.limpar_campos
        )
        btn_limpar.grid(row=0, column=2, padx=10)
        
        # Frame de busca e filtro
        frame_busca = tk.LabelFrame(
            frame_principal,
            text="  Busca e Filtro  ",
            font=("Arial", 11, "bold"),
            bg="#EAEAEA",
            fg="#333333",
            padx=20,
            pady=15
        )
        frame_busca.pack(pady=10, padx=20, fill='x')

        # Campo de busca
        tk.Label(
            frame_busca,
            text="Termo de Busca:",
            font=("Arial", 10),
            bg="#ffffff"
        ).grid(row=0, column=0, sticky='w', pady=8, padx=5)

        self.entry_busca = tk.Entry(frame_busca, width=30, font=("Arial", 10))
        self.entry_busca.grid(row=0, column=1, pady=8, padx=5)

        # Campo de filtro
        tk.Label(
            frame_busca,
            text="Filtrar por:",
            font=("Arial", 10),
            bg="#ffffff"
        ).grid(row=0, column=2, sticky='w', pady=8, padx=5)

        campos_busca = ["Nome", "Tipo", "Data de Adição"]
        self.campo_busca_var = tk.StringVar(frame_busca)
        self.campo_busca_var.set(campos_busca[0]) # Valor padrão

        self.combo_campo_busca = ttk.Combobox(
            frame_busca,
            textvariable=self.campo_busca_var,
            values=campos_busca,
            state="readonly",
            width=15,
            font=("Arial", 10)
        )
        self.combo_campo_busca.grid(row=0, column=3, pady=8, padx=5)

        # Botão de busca
        self.btn_buscar = tk.Button(
            frame_busca,
            text="🔍 Buscar",
            font=("Arial", 10, "bold"),
            bg="#2196F3",
            fg="white",
            cursor="hand2",
            command=self.buscar_estoque
        )
        self.btn_buscar.grid(row=0, column=4, padx=10)

        # Botão de limpar busca
        self.btn_limpar_busca = tk.Button(
            frame_busca,
            text="❌ Limpar",
            font=("Arial", 10, "bold"),
            bg="#E53935",
            fg="white",
            cursor="hand2",
            command=self.limpar_busca
        )
        self.btn_limpar_busca.grid(row=0, column=5, padx=10)
        
        # Lista de estoque atual
        frame_lista = tk.LabelFrame(
            frame_principal,
            text="  Estoque Atual  ",
            font=("Arial", 11, "bold"),
            bg="#EAEAEA",
            fg="#333333",
            padx=10,
            pady=10
        )
        frame_lista.pack(pady=10, padx=20, fill='both', expand=True)
        
        # Scrollbar
        scrollbar = tk.Scrollbar(frame_lista)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.listbox_estoque = tk.Listbox(
            frame_lista,
            font=("Courier", 10),
            yscrollcommand=scrollbar.set,
            height=10
        )
        self.listbox_estoque.pack(fill='both', expand=True)
        scrollbar.config(command=self.listbox_estoque.yview)
        
        self.atualizar_lista_gerenciamento()
    
    def adicionar_item(self):
        """Adiciona um item ao estoque"""
        nome = self.entry_nome.get().strip()
        tipo = self.entry_tipo.get().strip()
        
        try:
            quantidade = int(self.entry_quantidade.get())
            if quantidade <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("Erro", "Digite um número válido e positivo para a quantidade!")
            return
        
        if not nome:
            messagebox.showerror("Erro", "O nome do produto não pode estar vazio!")
            return
        
        if not tipo:
            messagebox.showerror("Erro", "O tipo do produto não pode estar vazio!")
            return
        
        adicionar_item_bd(nome, tipo, quantidade)
        messagebox.showinfo("Sucesso", f"{quantidade} unidade(s) de '{nome}' adicionada(s) ao estoque!")
        
        self.limpar_campos()
        self.buscar_estoque() # Atualiza a lista mantendo o filtro atual
        self.atualizar_estatisticas()
        self.atualizar_relatorio()
    
    def remover_item(self):
        """Remove um item do estoque"""
        nome = self.entry_nome.get().strip()
        
        try:
            quantidade = int(self.entry_quantidade.get())
            if quantidade <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("Erro", "Digite um número válido e positivo para a quantidade!")
            return
        
        if not nome:
            messagebox.showerror("Erro", "O nome do produto não pode estar vazio!")
            return
        
        sucesso, mensagem = remover_item_bd(nome, quantidade)
        
        if sucesso:
            messagebox.showinfo("Sucesso", f"{quantidade} unidade(s) de '{nome}' removida(s) do estoque!")
            self.limpar_campos()
            self.buscar_estoque() # Atualiza a lista mantendo o filtro atual
            self.atualizar_estatisticas()
            self.atualizar_relatorio()
        else:
            messagebox.showerror("Erro", mensagem)
    
    def limpar_campos(self):
        """Limpa os campos de entrada"""
        self.entry_nome.delete(0, tk.END)
        self.entry_tipo.delete(0, tk.END)
        self.entry_quantidade.delete(0, tk.END)
    
    def buscar_estoque(self):
        """Executa a busca e atualiza a lista de estoque"""
        self.atualizar_lista_gerenciamento(
            termo_busca=self.entry_busca.get().strip(),
            campo_busca=self.campo_busca_var.get()
        )

    def limpar_busca(self):
        """Limpa o campo de busca e exibe o estoque completo"""
        self.entry_busca.delete(0, tk.END)
        self.atualizar_lista_gerenciamento()

    def atualizar_lista_gerenciamento(self, termo_busca="", campo_busca="Nome"):
        """Atualiza a lista de estoque na aba de gerenciamento"""
        self.listbox_estoque.delete(0, tk.END)
        
        # Mapeia o nome do campo da interface para o nome da coluna no DB
        mapa_campos = {
            "Nome": "nome",
            "Tipo": "tipo",
            "Data de Adição": "data_adicao"
        }
        
        coluna_db = mapa_campos.get(campo_busca, "nome")
        
        itens = obter_estoque(termo_busca=termo_busca, campo_busca=coluna_db)
        
        if not itens:
            self.listbox_estoque.insert(tk.END, "  Nenhum item encontrado com o filtro aplicado.")
        else:
            for item in itens:
                # Verificação de segurança para garantir que a tupla tem todos os campos
                if len(item) >= 6:
                    texto = f"  {item[1]:<25} | Qtd: {item[3]:<6} | Tipo: {item[2]}"
                    self.listbox_estoque.insert(tk.END, texto)
    
    # ==================== ABA RELATÓRIOS ====================
    
    def criar_aba_relatorios(self):
        """Cria a aba de relatórios detalhados"""
        aba_relatorios = ttk.Frame(self.notebook)
        self.notebook.add(aba_relatorios, text="  📊 Relatórios  ")
        
        # Frame principal
        frame_principal = tk.Frame(aba_relatorios, bg="#ffffff")
        frame_principal.pack(fill='both', expand=True, padx=20, pady=20)
        
        # Título
        titulo = tk.Label(
            frame_principal,
            text="Relatórios Detalhados do Estoque",
            font=("Arial", 18, "bold"),
            bg="#ffffff",
            fg="#333333"
        )
        titulo.pack(pady=15)
        
        # Botão de atualizar
        btn_atualizar = tk.Button(
            frame_principal,
            text="🔄 Atualizar Relatório",
            font=("Arial", 10, "bold"),
            bg="#2196F3",
            fg="white",
            cursor="hand2",
            command=self.atualizar_relatorio
        )
        btn_atualizar.pack(pady=10)
        
        # Frame da tabela
        frame_tabela = tk.Frame(frame_principal, bg="#ffffff")
        frame_tabela.pack(fill='both', expand=True, pady=10)
        
        # Scrollbars
        scrollbar_y = tk.Scrollbar(frame_tabela)
        scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
        
        scrollbar_x = tk.Scrollbar(frame_tabela, orient=tk.HORIZONTAL)
        scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)
        
        # Treeview para relatório
        colunas = ("ID", "Nome", "Tipo", "Quantidade", "Data Adição", "Última Atualização")
        self.tree_relatorio = ttk.Treeview(
            frame_tabela,
            columns=colunas,
            show='headings',
            yscrollcommand=scrollbar_y.set,
            xscrollcommand=scrollbar_x.set,
            height=15
        )
        
        scrollbar_y.config(command=self.tree_relatorio.yview)
        scrollbar_x.config(command=self.tree_relatorio.xview)
        
        # Configurar colunas
        self.tree_relatorio.heading("ID", text="ID")
        self.tree_relatorio.heading("Nome", text="Nome do Produto")
        self.tree_relatorio.heading("Tipo", text="Tipo")
        self.tree_relatorio.heading("Quantidade", text="Quantidade")
        self.tree_relatorio.heading("Data Adição", text="Data de Adição")
        self.tree_relatorio.heading("Última Atualização", text="Última Atualização")
        
        self.tree_relatorio.column("ID", width=50, anchor='center')
        self.tree_relatorio.column("Nome", width=200, anchor='w')
        self.tree_relatorio.column("Tipo", width=150, anchor='w')
        self.tree_relatorio.column("Quantidade", width=100, anchor='center')
        self.tree_relatorio.column("Data Adição", width=150, anchor='center')
        self.tree_relatorio.column("Última Atualização", width=150, anchor='center')
        
        self.tree_relatorio.pack(fill='both', expand=True)
        
        # Estilo alternado nas linhas
        self.tree_relatorio.tag_configure('oddrow', background='#F9F9F9')
        self.tree_relatorio.tag_configure('evenrow', background='#ffffff')
        
        self.atualizar_relatorio()
    
    def atualizar_relatorio(self):
        """Atualiza a tabela de relatórios"""
        # Limpar tabela
        for item in self.tree_relatorio.get_children():
            self.tree_relatorio.delete(item)
        
        # Obter dados
        itens = obter_estoque()
        
        # Inserir dados
        for idx, item in enumerate(itens):
            # Verificação de segurança para garantir que a tupla tem todos os campos
            if len(item) >= 6:
                tag = 'evenrow' if idx % 2 == 0 else 'oddrow'
                self.tree_relatorio.insert(
                    '',
                    'end',
                    values=(item[0], item[1], item[2], item[3], item[4], item[5]),
                    tags=(tag,)
                )
    
    def atualizar_estatisticas(self):
        """Atualiza as estatísticas na aba inicial"""
        stats = obter_estatisticas()
        self.card_tipos.config(text=str(stats['total_tipos']))
        self.card_itens.config(text=str(stats['total_itens']))

# ==================== EXECUÇÃO ====================

if __name__ == "__main__":
    root = tk.Tk()
    app = GerenciadorEstoque(root)
    root.mainloop()